## SATCEN-01-01-01 Sentinel-2 Vegetation and Water Thematic Index application

This notebook takes a Sentinel-2 Level 1 product and generates the following results: 

* Atmospheric correction and cloud flagging (obtained via Sen2Corr) (TBC)
* All bands resampled to maximum resolution (10 m)
* Subset on AOI
* Compute the cloud coverage in the AOI and reject if above 20%
* Compute the NDVI and NDWI indices


In [1]:
import os
import sys
import numpy as np
import snappy
import dateutil.parser as parser
import gc
from datetime import datetime
import gdalnumeric
import osr

import cioppy
ciop = cioppy.Cioppy()

from shapely.wkt import loads
import lxml.etree as etree



In [2]:
service = dict([('title', 'NDVI-NDWI & Cloud Coverage Filtering'),
                ('abstract', 'Sentinel-2 NDVI NDWI'),
                ('id', 'ewf-satcen-01-01-01')])

In [3]:
wkt = dict([('id', 'aoi_wkt'),
               ('value','POLYGON ((21.29611111111111 39.58638888888889, 21.29611111111111 41.032, 19.89788888888889 41.032, 19.89788888888889 39.58638888888889, 21.29611111111111 39.58638888888889))'),
               ('title', 'Area of interest wkt'),
               ('abstract', 'Area of interest wkt')])


In [4]:
data_path = '/workspace/data'

In [5]:

#input_identifier ='S2B_MSIL2A_20180522T093029_N0207_R136_T34TDL_20180522T121543'
input_identifier ='S2A_MSIL2A_20180527T093041_N0206_R136_T34TDK_20180527T113215'


In [6]:
#input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20180522T093029_N0207_R136_T34TDL_20180522T121543'
input_reference = 'https://catalog.terradue.com//better-common-00001/series/results/search?uid=06cd99b5366608ae7966044f7b9dfb9db24b930d'


#### check if intersection with aoi_wkt exists

In [7]:
ciop = cioppy.Cioppy()
product_polygon = ciop.search(end_point=input_reference, 
                              params=[], 
                              output_fields='wkt',
                              model='GeoTime')

geom1 = loads(product_polygon[0]['wkt'])
geom2 = loads(wkt['value'])

if not geom1.intersects(geom2):
    raise ValueError('Area of Interest has no intersection with selected product.')



### check if u have MSIL2A on the mentioned path (BETTER-COMMON products to be checked)

In [8]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL2A.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)
width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()


In [9]:
product_date = parser.parse(product.getStartTime().toString()).date()

In [10]:
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)

#### If the product lacks B8, terminate the calculation

In [11]:
if not 'B8' in band_names :    
    raise ValueError('The selected Product has No Band B8.')

#### Resample the product to 10m resolution

In [12]:
reference_band = 'B4'
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')
parameters = HashMap()
parameters.put('referenceBand', reference_band)
product = snappy.GPF.createProduct('Resample', parameters, product)
   

#### Pixle Flag Expresion


In [13]:
flag_expr = dict([('id', 'flag_expr'),
               ('value',  'saturated_l1a_B4 or scl_water'),
               ('title', 'Flag expression for pixel exclusion'),
               ('abstract', 'Flag expression for pixel exclusion (e.g. saturated_l1a_B4 will exclude pixels having the flag saturated_l1a_B4 set)')])

## ndvi & ndwi computation

In [14]:

if not flag_expr['value'] :

        ndvi_expr = '(B8 + B4) != 0 ? 10000 + ((B8 - B4)/(B8 + B4)) * 10000 : 30000'
        ndwi_expr='(B8+B11)!=0? 10000+((B8-B11)/(B8+B11))*10000 :30000'

        
else:
   
    ndvi_expr = '! %s and (B8 + B4) != 0 ? 10000 + ((B8 - B4)/(B8 + B4)) * 10000 : 30000' % flag_expr['value']
    ndwi_expr = '! %s and (B8 + B11) != 0 ? 10000 + ((B8 - B11)/(B8 + B11)) * 10000 : 30000' % flag_expr['value']


In [15]:

displayed_Bands=['ndwi','ndvi','quality_cloud_confidence','quality_snow_confidence', 'quality_scene_classification', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

print(len(displayed_Bands))
print(displayed_Bands)


15
['ndwi', 'ndvi', 'quality_cloud_confidence', 'quality_snow_confidence', 'quality_scene_classification', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']


## Cloud Coverage Analysis

In [16]:
percentage_threshold = dict([('id', 'percentage_threshold'),
                           ('value', '20.0'),
                           ('title', 'Cloud percentage threshold'),
                           ('abstract', 'Cloud percentage threshold')])

In [17]:
HashMap = snappy.jpy.get_type('java.util.HashMap')

BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBand0 = BandDescriptor()
targetBand0.name = 'cloud_mask'
targetBand0.type = 'uint16'
targetBand0.expression = 'opaque_clouds_10m'


targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
targetBands[0] = targetBand0

 
parameters = HashMap()
parameters.put('targetBands', targetBands)

cloud_mask = snappy.GPF.createProduct('BandMaths', parameters, product)


In [18]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])
parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
cloud_mask_geo = snappy.GPF.createProduct('Subset', parameters, cloud_mask)
mask_geo_output_name = '%s_MASK_10.tif' %name
snappy.ProductIO.writeProduct(cloud_mask_geo, mask_geo_output_name,'GeoTIFF')

In [19]:
raster_file = gdalnumeric.LoadFile(mask_geo_output_name)
print raster_file.min(), raster_file.max()
pixel_count_cloud_geo = (raster_file == 255).sum()  # for pixel value = 1
print pixel_count_cloud_geo
cloud_percent =  float(pixel_count_cloud_geo) / float(raster_file.size) * 100.0

0 0
0


In [20]:
cloud_percent

0.0

## Creating Geotif for Target Bands : B1/B12 & ndvi & ndwi & Cloud Mask


In [21]:
if cloud_percent < float(percentage_threshold['value']):
    
    snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

    HashMap = snappy.jpy.get_type('java.util.HashMap')

    WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')
    
    geom = WKTReader().read(wkt['value'])
    
    geotiffs = []
    targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(displayed_Bands))
    
    for band in displayed_Bands:
        
        
        HashMap = snappy.jpy.get_type('java.util.HashMap')
        BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

        parameters = HashMap()
        parameters.put('copyMetadata', True)
        
        
        if band == 'ndvi' or band == 'ndwi':
            
            targetBand = BandDescriptor()
            targetBand.name = band
            targetBand.type = 'uint16'

            if band == 'ndvi':
                targetBand.expression = ndvi_expr
            elif band == 'ndwi':
                targetBand.expression = ndwi_expr
            
            targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', 1)
            targetBands[0] = targetBand
            parameters.put('targetBands', targetBands)
            subset = snappy.GPF.createProduct('BandMaths', parameters, product)

        else:
            parameters.put('sourceBands', band)
            subset = snappy.GPF.createProduct('Subset', parameters, product)
        

        parameters = HashMap()
        parameters.put('copyMetadata', True)
        parameters.put('geoRegion', geom)

        subset_geo = snappy.GPF.createProduct('Subset', parameters, subset)

        subsets.append(subset_geo)

        output_name = '%s_PA_CROP_%s.tif' % (name, band)
        geotiffs.append(output_name)

        snappy.ProductIO.writeProduct(subset_geo, output_name, 'GeoTIFF-BigTIFF')
else:
    raise ValueError('Selected product cloud coverage percentage is  %s >= %s' %(cloud_percent,percentage_threshold['value']))

In [22]:
product = None
gc.collect()

0

### EOP Metadata

In [23]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % opt)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    orbit_number.text = metadata['orbitNumber']
    wrs_longitude_grid.text = metadata['wrsLongitudeGrid']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)




## Create EOP XML file

In [24]:
search = ciop.search(end_point=input_reference,
                     params=[],
                     output_fields='enclosure,identifier,startdate,enddate,wkt,orbitNumber,orbitDirection,wrsLongitudeGrid',
                     model='EOP')

search[0]['wkt'] = wkt['value']

for output_name in geotiffs:
    
    search[0]['identifier'] = output_name.replace('.tif', '')
    eop_xml = output_name+'.xml'
    with open(eop_xml, 'wb') as file:
        file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        file.write(eop_metadata(search[0]))



### Removing tifs won't be published

In [ ]:
produced_tifs = [f for f in os.listdir(base_path) if f.endswith('.tif')]

for f in [item for item in produced_tifs if item not in geotiffs]:
    os.remove(f)